# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [37]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [38]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [39]:
%%bigquery df --project slu-student-397219
SELECT
  COUNT(*) as total_rows
FROM `slu-student-397219.tanneru_dataset_csv.final_patien_payer_details`

Query is running:   0%|          |

Downloading:   0%|          |

In [40]:
df

,total_rows
0,3801


In [41]:
project_id = 'slu-student-397219'

### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [42]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT
    COUNT(*) as total
  FROM `slu-student-397219.tanneru_dataset_csv.final_patien_payer_details`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `slu-student-397219.tanneru_dataset_csv.final_patien_payer_details`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 3801 rows


In [43]:
df = pd.DataFrame(df)
df

,Patient_Id,Full_Name,BIRTHDATE,BIRTHPLACE,Location,GENDER,HEALTHCARE_COVERAGE,HEALTHCARE_EXPENSES,DRIVERS,ADDRESS,...,AMOUNT_UNCOVERED,COVERED_ENCOUNTERS,COVERED_IMMUNIZATIONS,COVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_ENCOUNTERS,UNCOVERED_IMMUNIZATIONS,UNCOVERED_MEDICATIONS,UNCOVERED_PROCEDURES,Payer_Revenue
0,03172f6e-fb21-4770-8eef-513730174ab7,Kala987 Rutherford999,1961-03-14,Clinton Massachusetts US,None,F,344.12,29303.22,None,343 Kessler Lodge Apt 51,...,10129408.90,0,0,0,0,10116,3644,5465,8281,0.0
1,2c71dd97-7085-416a-aa07-d675bbe3adf2,Samatha845 Mueller846,1941-02-14,Lenox Massachusetts US,None,F,145918.88,1694231.71,S99975437,531 Collier Wynd,...,10129408.90,0,0,0,0,10116,3644,5465,8281,0.0
2,42f5948f-7a5f-417e-838c-e4ff331deadf,Darren774 Keebler762,2008-04-25,Sharon Massachusetts US,None,M,516.64,19255.88,None,1072 Hagenes Crossing,...,10129408.90,0,0,0,0,10116,3644,5465,8281,0.0
3,0b6a83ae-fcb1-4b75-9ffa-d52898167d66,Shirley182 Wunsch504,1989-08-05,Plymouth Massachusetts US,None,M,0.00,13891.61,S99976161,380 Schamberger Mall,...,10129408.90,0,0,0,0,10116,3644,5465,8281,0.0
4,058590f8-c0f3-40c1-ac80-246437499d5d,Rubye299 Upton904,1981-02-05,Boston Massachusetts US,None,F,2523.59,63274.83,S99970594,624 Bailey Forge Suite 81,...,10129408.90,0,0,0,0,10116,3644,5465,8281,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,efa20606-8972-4bd7-823e-2011fe9a54e1,Milissa240 Weimann465,1923-05-15,Revere Massachusetts US,County : Hampden County City: West Springfield...,F,13182.04,1305867.75,S99961825,154 Leannon Gardens Suite 18,...,652470.00,11484,6682,0,12586,1873,156,4044,801,141468000.0
2002,efa20606-8972-4bd7-823e-2011fe9a54e1,Milissa240 Weimann465,1923-05-15,Revere Massachusetts US,County : Hampden County City: West Springfield...,F,13182.04,1305867.75,S99961825,154 Leannon Gardens Suite 18,...,635040.00,11971,6661,0,12302,1745,164,4408,592,154184100.0
2003,92b4bd25-f752-49fe-b248-a3e4b0e89d2a,Armand155 Grady603,1918-02-16,Easthampton Massachusetts US,County : Worcester County City: North Brookfie...,M,9775.49,1681224.24,S99951461,687 Heidenreich Loaf,...,890394.37,11072,2557,13471,7383,0,0,0,0,7122250.0
2004,92b4bd25-f752-49fe-b248-a3e4b0e89d2a,Armand155 Grady603,1918-02-16,Easthampton Massachusetts US,County : Worcester County City: North Brookfie...,M,9775.49,1681224.24,S99951461,687 Heidenreich Loaf,...,635040.00,11971,6661,0,12302,1745,164,4408,592,154184100.0


In [44]:
df.columns

Index(['Patient_Id', 'Full_Name', 'BIRTHDATE', 'BIRTHPLACE', 'Location',
       'GENDER', 'HEALTHCARE_COVERAGE', 'HEALTHCARE_EXPENSES', 'DRIVERS',
       'ADDRESS', 'ETHNICITY', 'RACE', 'SSN', 'SUFFIX', 'Payer_Id',
       'Payer_Name', 'Payer_Phone', 'AMOUNT_COVERED', 'AMOUNT_UNCOVERED',
       'COVERED_ENCOUNTERS', 'COVERED_IMMUNIZATIONS', 'COVERED_MEDICATIONS',
       'COVERED_PROCEDURES', 'UNCOVERED_ENCOUNTERS', 'UNCOVERED_IMMUNIZATIONS',
       'UNCOVERED_MEDICATIONS', 'UNCOVERED_PROCEDURES', 'Payer_Revenue'],
      dtype='object')

### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [45]:
#estimating the average cost of healthcare by ethnicity and gender
agg_df = df.groupby(['GENDER', 'ETHNICITY']).agg({'HEALTHCARE_EXPENSES': 'mean'}).reset_index()
agg_df

,GENDER,ETHNICITY,HEALTHCARE_EXPENSES
0,F,hispanic,9.731732e+05
1,F,nonhispanic,1.100279e+06
2,M,hispanic,7.232494e+05
3,M,nonhispanic,1.004175e+06


In [46]:
#Total healthcare costs broken down by payer
agg_df_payer = df.groupby(['Payer_Id', 'Payer_Name']).agg({
    'AMOUNT_COVERED': 'sum',
    'AMOUNT_UNCOVERED': 'sum'
}).reset_index()
agg_df_payer

,Payer_Id,Payer_Name,AMOUNT_COVERED,AMOUNT_UNCOVERED
0,047f6ec3-6215-35eb-9608-f9dda363a44c,Cigna Health,1.619198e+08,1.130371e+08
1,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,Anthem,8.114931e+07,3.988523e+07
2,4d71f845-a6a9-3c39-b242-14d25ef86a8d,Aetna,1.332062e+08,1.050477e+08
3,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,UnitedHealthcare,1.506283e+08,1.370285e+08
4,6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield,5.456018e+08,2.848613e+08
5,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,2.713198e+09,1.353966e+09
6,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,5.850766e+08,1.095185e+08
7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0.000000e+00,1.023070e+09
8,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,1.275092e+06,1.075048e+06
9,d47b3510-2895-3b70-9897-342d681c769d,Humana,1.183003e+08,1.390177e+08


In [47]:
# dividing the total number of vaccinations and encounters by gender
agg_df_gender = df.groupby('GENDER').agg({
    'COVERED_ENCOUNTERS': 'sum',
    'COVERED_IMMUNIZATIONS': 'sum',
    'UNCOVERED_ENCOUNTERS': 'sum',
    'UNCOVERED_IMMUNIZATIONS': 'sum'
}).reset_index()
agg_df_gender

,GENDER,COVERED_ENCOUNTERS,COVERED_IMMUNIZATIONS,UNCOVERED_ENCOUNTERS,UNCOVERED_IMMUNIZATIONS
0,F,29869632,10945865,2674615,377552
1,M,3915364,1741481,732812,127429


### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [48]:
with pd.ExcelWriter('aggregated_data.xlsx', engine='openpyxl') as writer:
  agg_df.to_excel(writer, sheet_name='Healthcare_Expenses_By_Gender_Ethnicity', index=False)
  agg_df_payer.to_excel(writer, sheet_name='Total_Healthcare_Expenses_By_Payer', index=False)
  agg_df_gender.to_excel(writer, sheet_name='Encounters_and_Immunizations_By_Gender', index=False)


/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


---

## Submitting Your Work

Submit your work as usual